In [1]:
# Cell 1: Install Libraries
!pip install open_clip_torch==2.23.0 transformers==4.35.2 matplotlib -q
!pip install scikit-learn -q
!pip install nltk -q # Ensure nltk is installed for text processing and metrics
!pip install rouge -q # Ensure rouge is installed for ROUGE score
!pip install evaluate -q # A HuggingFace library for easier metric calculation

# For METEOR score, often requires specific installation or a separate script
# if meteor_score is not part of the 'evaluate' library or nltk directly.
# If you used 'meteor_score.py' previously, ensure it's accessible.
# For consistency, it's good to use a unified metrics library if possible.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 111.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.1.2 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.35.2 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.7 MB/s eta 0:00:00


In [2]:
# Cell 2: Mount Google Drive and Define Global Configuration

import os
import pandas as pd
from PIL import Image
from tqdm.notebook import tqdm
import torch
import open_clip # from open_clip_torch
import numpy as np
import torch.cuda.amp as amp # <--- ADD THIS LINE

# Mount your drive
from google.colab import drive
drive.mount('/content/drive')

# -----------------------------
# Configuration
# -----------------------------
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Running on device: {DEVICE}")
if DEVICE == 'cuda':
    print(f"CUDA Device: {torch.cuda.get_device_name(0)}")
else:
    print("No GPU available, running on CPU. This will be slower.")

# Paths to your CSV and images in Google Drive
# In Cell 1, update these paths
INPUT_CSV_FILES = [
    "/content/drive/MyDrive/Colab Notebooks/Final_Train_Data.csv", # <--- CHANGE THIS LINE
    "/content/drive/MyDrive/Colab Notebooks/Final_CV_Data.csv",    # <--- CHANGE THIS LINE
    "/content/drive/MyDrive/Colab Notebooks/Final_Test_Data.csv"    # <--- CHANGE THIS LINE
]
IMAGE_BASE_DIR = "/content/drive/MyDrive/Colab Notebooks /NLMCXR_png" # <--- ALSO UPDATE THIS IF NLMCXR_png is there
# --- CHECK THIS PATH CAREFULLY AGAINST YOUR GOOGLE DRIVE ---
# <--- Adjusted based on common setup/screenshot
# --- END PATH CHECK ---

# --- IMPORTANT: Set the model identifier for output filenames ---
# Based on your files, we will use the BioMedCLIP model locally.
MODEL_IDENTIFIER_FOR_OUTPUT = "biomedclip_vitb16"

# This variable needs to be consistently defined for Cell 3
BIOMEDCLIP_LOCAL_MODEL_DIR = "/content/drive/MyDrive/BiomedCLIP_Model_Files" # <--- Ensure this is correct
os.makedirs(BIOMEDCLIP_LOCAL_MODEL_DIR, exist_ok=True) # Ensure it exists if you upload manually

/usr/local/lib/python3.12/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.12/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.12/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


Mounted at /content/drive
Running on device: cuda
CUDA Device: Tesla T4


In [3]:
# Cell 3: Load the BioMedCLIP Model

# Ensure BIOMEDCLIP_LOCAL_MODEL_DIR is set (it should be from Cell 2 now)
if 'BIOMEDCLIP_LOCAL_MODEL_DIR' not in locals():
    # Fallback if cell 2 was skipped, but ideally set there.
    BIOMEDCLIP_LOCAL_MODEL_DIR = "/content/drive/MyDrive/BiomedCLIP_Model_Files"

print(f"\nLoading BioMedCLIP model from local directory: {BIOMEDCLIP_LOCAL_MODEL_DIR} using open_clip...")
try:
    # Use torch.float16 for `precision` if DEVICE is 'cuda', otherwise float32.
    # This ensures consistency with the model.half() call below.
    model, _, preprocess = open_clip.create_model_and_transforms(
        model_name="ViT-B-16",
        pretrained=None, # Important: Set to None as you're loading from local cache_dir
        cache_dir=BIOMEDCLIP_LOCAL_MODEL_DIR,
        precision='fp16' if DEVICE == 'cuda' else 'fp32', # Use the global DEVICE variable
        device=DEVICE
    )

    # --- IMPORTANT: Convert model to half precision (fp16) for GPU if not already ---
    # open_clip's create_model_and_transforms' `precision` argument primarily controls how weights are loaded.
    # `model.half()` explicitly converts the *entire model* to FP16, which is crucial for performance on GPUs.
    if DEVICE == 'cuda':
        model = model.half() # Convert model to half precision (fp16)
        # Check the dtype of a specific parameter if you want to verify
        if hasattr(model, 'visual') and hasattr(model.visual, 'conv1') and hasattr(model.visual.conv1, 'weight'):
            print(f"Model (visual part) converted to {model.visual.conv1.weight.dtype} for CUDA acceleration.")
        else:
            print("Model converted to half precision (fp16) for CUDA acceleration.")
    # --- END IMPORTANT CHANGE ---

    tokenizer = open_clip.get_tokenizer("ViT-B-16")
    model.eval() # Set model to evaluation mode (disables dropout, batch norm updates)
    print("BioMedCLIP model and preprocessor loaded successfully from local drive!")

except Exception as e:
    print(f"Error loading BioMedCLIP with open_clip from local drive: {e}")
    print(f"Please ensure the directory '{BIOMEDCLIP_LOCAL_MODEL_DIR}' exists and contains all required files (e.g., config.json, pytorch_model.bin).")
    raise # Re-raise the exception to stop execution if model loading fails


Loading BioMedCLIP model from local directory: /content/drive/MyDrive/BiomedCLIP_Model_Files using open_clip...
Model (visual part) converted to torch.float16 for CUDA acceleration.
BioMedCLIP model and preprocessor loaded successfully from local drive!


In [4]:
# Cell 4: Define Feature Extraction Function

# -----------------------------
# Feature Extraction Function
# -----------------------------
def extract_features(image_path, model, preprocess, device):
    try:
        if not os.path.exists(image_path):
            print(f"Warning: Image file not found at {image_path}. Skipping.")
            return None

        img = Image.open(image_path).convert('RGB')

        # Preprocess the image using open_clip's preprocess function
        # preprocess() typically outputs float32, which is fine for autocast
        image_input = preprocess(img).unsqueeze(0).to(device)

        # --- NEW CHANGE HERE: Use autocast for mixed precision inference ---
        with torch.no_grad():
           if device == 'cuda':
                with torch.amp.autocast(device_type='cuda'): # Recommended syntax
                    image_features = model.encode_image(image_input)
           else:
                image_features = model.encode_image(image_input)

        # Convert to float32 and move to CPU, then to numpy
        return image_features.float().cpu().numpy().squeeze()
    except Exception as e:
        print(f"Failed on {image_path}: {e}")
        return None

In [ ]:
# @title 5. Process CSV Files and Save Features

tqdm.pandas()

print(f"\nStarting feature extraction process for all CSV files...")

for csv_path in INPUT_CSV_FILES:
    df = pd.read_csv(csv_path)
    print(f"\nProcessing {csv_path} with {len(df)} rows...")

    if 'Image1' not in df.columns or 'Image2' not in df.columns:
        print(f"Warning: Missing 'Image1' or 'Image2' columns in {csv_path}. Skipping...")
        continue

    df['image_feat_img1'] = df['Image1'].progress_apply(
        lambda x: extract_features(
            os.path.join(IMAGE_BASE_DIR, x.replace('NLMCXR_png/', '', 1) if x and x.startswith('NLMCXR_png/') else x), # Added 'if x' for robustness
            model, preprocess, DEVICE
        ) if pd.notna(x) else None
    )
    df['image_feat_img2'] = df['Image2'].progress_apply(
        lambda x: extract_features(
            os.path.join(IMAGE_BASE_DIR, x.replace('NLMCXR_png/', '', 1) if x and x.startswith('NLMCXR_png/') else x), # Added 'if x' for robustness
            model, preprocess, DEVICE
        ) if pd.notna(x) else None
    )

    output_path = csv_path.replace(".csv", f"_with_{MODEL_IDENTIFIER_FOR_OUTPUT}_features.pkl")
    df.to_pickle(output_path)
    print(f"Saved: {output_path}")

print(f"\n✅ All done. {MODEL_IDENTIFIER_FOR_OUTPUT} features saved.")

# ----------------------------------------------------
# Verification Block: Loading and checking the saved features
# 🚨🚨🚨 CRITICAL: FIX THESE PATHS TO MATCH YOUR SAVED FILES 🚨🚨🚨
# ----------------------------------------------------
import pandas as pd
import os
import torch
import numpy as np

print("\n--- Verifying Loaded Processed Data ---")

# These paths MUST match the output filenames generated by the loop above.
# They are derived from INPUT_CSV_FILES and MODEL_IDENTIFIER_FOR_OUTPUT.
base_dir_of_csvs = os.path.dirname(INPUT_CSV_FILES[0]) # Assuming all CSVs are in the same folder

TRAIN_PKL_PATH = os.path.join(base_dir_of_csvs, f"Final_Train_Data_with_{MODEL_IDENTIFIER_FOR_OUTPUT}_features.pkl")
CV_PKL_PATH = os.path.join(base_dir_of_csvs, f"Final_CV_Data_with_{MODEL_IDENTIFIER_FOR_OUTPUT}_features.pkl")
TEST_PKL_PATH = os.path.join(base_dir_of_csvs, f"Final_Test_Data_with_{MODEL_IDENTIFIER_FOR_OUTPUT}_features.pkl")

print(f"Attempting to load from: \nTrain: {TRAIN_PKL_PATH}\nCV: {CV_PKL_PATH}\nTest: {TEST_PKL_PATH}")

try:
    df_train_features = pd.read_pickle(TRAIN_PKL_PATH)
    print(f"\nLoaded training data with features from: {TRAIN_PKL_PATH}")
    print(f"Shape of loaded training data: {df_train_features.shape}")
    print(f"Columns in training data: {df_train_features.columns.tolist()}")

    df_cv_features = pd.read_pickle(CV_PKL_PATH)
    print(f"\nLoaded CV data with features from: {CV_PKL_PATH}")
    print(f"Shape of loaded CV data: {df_cv_features.shape}")
    print(f"Columns in CV data: {df_cv_features.columns.tolist()}")

    df_test_features = pd.read_pickle(TEST_PKL_PATH)
    print(f"\nLoaded test data with features from: {TEST_PKL_PATH}")
    print(f"Shape of loaded test data: {df_test_features.shape}")
    print(f"Columns in test data: {df_test_features.columns.tolist()}")

    print("\nFirst 5 rows of training data with features:")
    print(df_train_features[['Image1', 'image_feat_img1', 'Image2', 'image_feat_img2']].head())

    print("\nChecking for any None values in feature columns:")
    print(f"None values in 'image_feat_img1' (Train): {df_train_features['image_feat_img1'].isnull().sum()}")
    print(f"None values in 'image_feat_img2' (Train): {df_train_features['image_feat_img2'].isnull().sum()}")
    print(f"None values in 'image_feat_img1' (CV): {df_cv_features['image_feat_img1'].isnull().sum()}")
    print(f"None values in 'image_feat_img2' (CV): {df_cv_features['image_feat_img2'].isnull().sum()}")
    print(f"None values in 'image_feat_img1' (Test): {df_test_features['image_feat_img1'].isnull().sum()}")
    print(f"None values in 'image_feat_img2' (Test): {df_test_features['image_feat_img2'].isnull().sum()}")

except FileNotFoundError as e:
    print(f"🚫 Error: A .pkl file was not found. Please ensure all paths in Cell 2 are correct and Cell 5 successfully completed the feature extraction process.")
    print(f"Details: {e}")
except Exception as e:
    print(f"🚫 An unexpected error occurred while loading or processing the pickle files: {e}")


Starting feature extraction process for all CSV files...

Processing /content/drive/MyDrive/Colab Notebooks/Final_Train_Data.csv with 2764 rows...


  0%|          | 0/2764 [00:00<?, ?it/s]

  0%|          | 0/2764 [00:00<?, ?it/s]

Saved: /content/drive/MyDrive/Colab Notebooks/Final_Train_Data_with_biomedclip_vitb16_features.pkl

Processing /content/drive/MyDrive/Colab Notebooks/Final_CV_Data.csv with 609 rows...


  0%|          | 0/609 [00:00<?, ?it/s]

  0%|          | 0/609 [00:00<?, ?it/s]

Saved: /content/drive/MyDrive/Colab Notebooks/Final_CV_Data_with_biomedclip_vitb16_features.pkl

Processing /content/drive/MyDrive/Colab Notebooks/Final_Test_Data.csv with 377 rows...


  0%|          | 0/377 [00:00<?, ?it/s]

  0%|          | 0/377 [00:00<?, ?it/s]

In [ ]:
import nbformat

path = "/content/drive/MyDrive/Colab Notebooks /BioMedCLIP_pipeline.ipynb"

nb = nbformat.read(path, as_version=4)

# Remove global widget metadata
if "widgets" in nb["metadata"]:
    del nb["metadata"]["widgets"]

# Clean metadata of each cell
for cell in nb["cells"]:
    if "metadata" in cell:
        # Remove widget metadata
        if "widgets" in cell["metadata"]:
            del cell["metadata"]["widgets"]

        # Remove unnecessary Colab metadata that breaks GitHub rendering
        for key in ["colab", "id", "outputId", "execution"]:
            if key in cell["metadata"]:
                del cell["metadata"][key]

        # Clean outputs metadata too
        if "outputs" in cell:
            for output in cell["outputs"]:
                if hasattr(output, "metadata") and "widgets" in output.metadata:
                    del output.metadata["widgets"]

# Write cleaned notebook
nbformat.write(nb, path)

print("Deep-clean complete. All widget metadata removed.")


In [ ]:
from google.colab import files
files.download("/content/drive/MyDrive/Colab Notebooks /BioMedCLIP_pipeline.ipynb")
